# simInvNB Using the GPU

## Crude Helper Function

In [2]:
using Distributions
using LinearAlgebra
using RCall

┌ Info: Precompiling RCall [6f49c342-dc21-5d91-9882-a32aef131414]
└ @ Base loading.jl:1260


In [3]:
function cov2cor(M)
    Mcor = copy(M)
    @inbounds for i in CartesianIndices(M)
        Mcor[i] = M[i] / sqrt(M[i[1], i[1]] * M[i[2], i[2]])
    end
    return Mcor
end

cov2cor (generic function with 1 method)

In [4]:
function rcor(d, ρ_constant=false)
    if ρ_constant
        ρ_temp = rand(Uniform(0.25, 0.90))
        Σ = fill(ρ_temp, (d, d))
        @inbounds for i = 1:d
            Σ[i,i] = 1.0
        end
    else
        tmp  = randn(d, d)
        mcov = tmp * tmp'
        Σ    = cov2cor(mcov)
    end
    return Σ
end     

rcor (generic function with 2 methods)

In [5]:
function rmvnb(d::Int)
    rates = rand(1:5, d)
    probs = rand(0.2:0.01:0.8, d)
    return collect((NegativeBinomial, (rate, prob)) for (rate, prob) in zip(rates, probs))
end

rmvnb (generic function with 1 method)

In [6]:
rmvnb(4)

4-element Array{Tuple{UnionAll,Tuple{Int64,Float64}},1}:
 (NegativeBinomial, (3, 0.75))
 (NegativeBinomial, (4, 0.31))
 (NegativeBinomial, (4, 0.44))
 (NegativeBinomial, (2, 0.43))

## Helper Functions

In [7]:
function convertCor(ρ, from, to)
    F = Dict(
        ("pearson", "pearson")  => A -> A,
        ("pearson", "spearman") => A -> (6 / pi) .* asin.(A ./ 2),
        ("pearson", "kendall")  => A -> (2 / pi) .* asin.(A),
        ("spearman", "pearson") => A -> 2 .* sin.(A .* (pi / 6)),
        ("spearman", "spearman")=> A -> A,
        ("spearman", "kendall") => A -> (2 / pi) .* asin.(2 .* sin.(A .* (pi / 6))),
        ("kendall", "pearson")  => A -> sin.(A .* (pi / 2)),
        ("kendal", "spearman")  => A -> (6 / pi) .* asin.(sin.(A .* (pi / 2)) ./ 2),
        ("kendall", "kendall")  => A -> A
    )
    tmp = F[(from, to)](ρ)
    @rput tmp
    return rcopy(R"as.matrix(Matrix::nearPD(tmp)$mat)")
end

convertCor (generic function with 1 method)

In [8]:
function adjustCor(ρ, params)
    d = size(ρ, 1)
    
    indexMat = rcopy(R"combn(1:$d, 2)")
    ρ_out = Matrix{Float64}(I, d, d)
    
    iter = [(j,i) for (j,i) in enumerate(eachcol(indexMat))]
    Threads.@threads for it in iter
        j = it[1]
        i = it[2]
        α₁ = params[:α][i[1]]
        α₂ = params[:α][i[2]]
        λ₁ = params[:λ][i[1]]
        λ₂ = params[:λ][i[2]]
        σ₁ = sqrt(params[:σ][i[1]])
        σ₂ = sqrt(params[:σ][i[2]])
        
        scaleFactor = sqrt(α₁ * α₂)
        ρ_tmp = ρ[i...]
        ρ_targetGamma = ρ_tmp * (σ₁ * σ₂) / (scaleFactor * λ₁ * λ₂)
        @inbounds ρ_out[i...] = ρ_out[reverse(i)...] = min(1, ρ_targetGamma)
    end
    
    eigenDecomp = eigen(ρ_out)
    if any(eigenDecomp.values .< 0)
        @rput ρ_out
        return cov2cor(rcopy(R"as.matrix(Matrix::nearPD(ρ_out)$mat)"))
    end
    
    return ρ_out
end

adjustCor (generic function with 1 method)

In [9]:
# margin = (Distribution, (param1, param2, ...))
function normal2marginal(x, margin)
    quantile.(margin[1](margin[2]...), cdf.(Normal(), x))
end

normal2marginal (generic function with 1 method)

## Simulate Negative Binomial

In [10]:
function rvec(n, ρ, margins, inputCorType)
    ρ = convertCor(ρ, inputCorType, "pearson")
    d = size(ρ, 1)

    mvn = rand(MultivariateNormal(zeros(d), ρ), n)'
    
    Threads.@threads for i in 1:d
        mvn[:,i] = normal2marginal(mvn[:,i], margins[i])
    end
    
    return mvn
end

rvec (generic function with 1 method)

# Testing

In [11]:
using Test

In [16]:
d = 1000
n = Int(1e4)
ρ = rcor(d);
margins = rmvnb(d)
s1 = rvec(n, ρ, margins, "pearson")
ρ_hat = cor(s1)

1000×1000 Array{Float64,2}:
  1.0         -4.96977e-5   -0.024831    …   0.0285552     0.00283257
 -4.96977e-5   1.0           0.00571509     -0.0145097    -0.0124817
 -0.024831     0.00571509    1.0             0.034235      0.0292596
 -0.0497791   -0.0339075    -0.0219188       0.0418519     0.0433473
  0.0235752   -0.0408207    -0.0200231       0.00686585   -0.0461368
  0.0106226    0.0409509     0.0151158   …   0.00155695    0.00591934
 -0.0401845    0.0353036     0.0375004      -0.0515039    -0.0425675
  0.00171626   0.00785709    0.042297        0.00620113    0.0350942
  0.0148377    0.0390678     0.0231779       0.0157224    -0.0472018
  0.0642118   -0.0418073     0.00427608     -0.00430518    0.0146081
  0.0212841    0.0141278    -0.00599587  …  -0.0147713     0.0342745
  0.0190852   -0.0456527    -0.0105069      -0.0301319     0.0120687
  0.00361333   0.0288592     0.0107117       0.0183042    -0.0563393
  ⋮                                      ⋱                
 -0.00309025  

In [24]:
ρ_hat[findall(abs.(ρ .- ρ_hat) .> 0.057)]

10-element Array{Float64,1}:
  0.023172562698905767
 -0.06654367278840025
  0.023172562698905767
  0.024909591158790406
 -0.04411580403870944
 -0.04411580403870944
  0.03776570102391063
 -0.06654367278840025
  0.024909591158790406
  0.03776570102391063

In [25]:
ρ[findall(abs.(ρ .- ρ_hat) .> 0.057)]

10-element Array{Float64,1}:
  0.08062146528208884
 -0.12518846580872178
  0.08062146528208884
  0.08285939616761884
 -0.10264689908870102
 -0.10264689908870102
  0.09589531683737997
 -0.12518846580872178
  0.08285939616761884
  0.09589531683737997

In [26]:
ρ[findall(sign.(ρ) .!= sign.(ρ_hat))]

115000-element Array{Float64,1}:
  0.020528137207175706
 -0.002194448917754211
 -0.0017940108441060379
 -7.407153228458754e-5
 -0.005841991493852829
 -0.00010122826581239139
 -0.0048664352143473035
 -0.0022737049324424736
  0.0068467882151009045
  0.0074696606755299535
  0.0007907738500876401
  0.0049114517071707315
 -0.0023494836781127043
  ⋮
 -0.007048756039653522
 -0.012191003425743351
  0.011549806133159023
 -0.005011244062609902
 -0.001597787042511401
  0.01014159800856057
 -0.019698935754295297
 -0.001477844570464791
 -0.0030952011437844163
 -0.007567789223555355
  0.002378409211574413
 -0.015829044559791266

In [27]:
ρ_hat[findall(sign.(ρ) .!= sign.(ρ_hat))]

115000-element Array{Float64,1}:
 -4.9697742279532574e-5
  0.012382250184789922
  0.008268549544674752
  0.0033378972604191886
  0.002205879293068132
  0.010955870571039139
  0.0028992657242496527
  0.014637972307079803
 -0.00785130400211254
 -0.007338384217526894
 -0.006631409199766768
 -0.01567699522372538
  0.019900988583533688
  ⋮
  0.0032251405641661656
  0.021029848201440295
 -4.8046450963770335e-5
  0.002493243640345325
  0.0010668062481008927
 -0.00037052031216175776
  0.0018088902872004703
  0.004522559601894479
  0.000666251306166924
  0.00970133127055489
 -0.002331775775725134
  0.005942665132980212